In [10]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

df = pd.read_csv("data/npf_train.csv")

df["event"] = df["class4"] != "nonevent"
df = df.drop(["class4"], axis = 1)
y = df["event"].values



#Remove columns that we don't want to use.
columns = df.columns.values.tolist()
remove = ["id", "date", "event"]
columns = [column for column in columns if column not in remove]

df = df[columns].apply(pd.to_numeric, errors='coerce')

#df = pd.to_numeric(df)
#df = df.apply(lambda x: abs(x))

#print(columns)

#for column in columns:
    #negative = df[column] < 0
    #if any(negative.values.tolist()):
        #remove.append(column)

#print(columns)

pipeline = Pipeline([
    ('minmax', MinMaxScaler()),
    ('model', MultinomialNB()),
])

#X = df[columns].values
#model = MultinomialNB()

#X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2)
#X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size= 0.75)

#model.fit(X_train, y_train)

<>:36: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
<>:37: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
<>:36: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
<>:37: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
C:\Users\Rasmus Marjoranta\AppData\Local\Temp\ipykernel_24316\3786446610.py:36: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
  ...     ('minmax', MinMaxScaler()),
C:\Users\Rasmus Marjoranta\AppData\Local\Temp\ipykernel_24316\3786446610.py:37: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
  ...     ('model', MultinomialNB()),
C:\Users\Rasmus Marjoranta\AppData\Local\Temp\ipykernel_24316\3786446610.py:36: SyntaxWarning: 'ellipsis' object is not callable; perhaps you missed a comma?
  ...     ('minmax', MinMaxScaler()),
C:\Users\Rasmus Marjoranta\AppData\Local\Temp\ipykernel_24316\3786446610

TypeError: 'ellipsis' object is not callable